# Analyse Sentimentale 

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import re
import json
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import pandas as pd 
from IPython.display import display
from transformers  import pipeline
import torch

c:\nlp\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_reviews(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        reviews = file.readlines()
    return reviews



In [3]:
# Charger les titres des films depuis le fichier JSON
def load_movie_titles(json_file):
    with open(json_file, 'r', encoding='utf-8') as file:
        movie_titles = json.load(file)
    return movie_titles



In [ ]:
def count_reviews(reviews, movie_titles):
    total_reviews = 0
    for line in reviews:
        line = line.strip()  

        if not line:
            continue
        if line.startswith("Film : "):
            title = line[len("Film : "):].strip() 
            if title in movie_titles:
                continue 
        
        total_reviews += 1

    return total_reviews

In [ ]:
file_path = 'critiques_films.txt'  # Chemin vers le fichier de critiques
json_file = r'C:\nlp\movie_titles.json'  

reviews = load_reviews(file_path)
movie_titles = load_movie_titles(json_file)
s
total_reviews = count_reviews(reviews, movie_titles)

# Afficher le nombre total de critiques
print(f"Nombre total de critiques : {total_reviews}")

Nombre total de critiques : 750


In [6]:
def preprocess_reviews(reviews, movie_titles):
    processed_reviews = []
    
    for review in reviews:
        review = review.strip()  # Enlever les espaces au début et à la fin de la ligne

        # Si la ligne est vide ou si la ligne est un titre de film, on l'ignore
        if not review or review.startswith("Film : "):
            continue

        # Sinon, on prétraite la critique
        review = review.lower()  # Mettre en minuscules
        review = re.sub(r'\d+', '', review)  # Supprimer les chiffres
        review = re.sub(r'[^\w\s]', '', review)  # Supprimer la ponctuation
        processed_reviews.append(review)
    
    return processed_reviews

# Appliquer le prétraitement
reviews_cleaned = preprocess_reviews(reviews, movie_titles)
print(f"Exemple de critique nettoyée : {reviews_cleaned[16]}")
print(f"Nombre de critiques nettoyées : {len(reviews_cleaned)}")

Exemple de critique nettoyée : on aurait pu sattendre de le part de spielberg à un manichéisme bien politiquement correct on a au contraire une oeuvre sobre dotée dun regard juste et réfléchi sur la nature humaine le réalisateur a réussi à éviter tous les écueils dans lesquels il aurait pourtant été facile de tomber  cest dautant plus admirable emouvant parce que brillant
Nombre de critiques nettoyées : 750


In [7]:
with open('movie_titles.json', 'r') as file:
    movie_titles = json.load(file)



# Créer un DataFrame avec les critiques
df = pd.DataFrame({'critique': reviews_cleaned})

# Assigner un titre de film à chaque critique. Répartir les 50 titres parmi les 750 critiques.
df['film_title'] = [movie_titles[i // (len(reviews_cleaned) // 50)] for i in range(len(reviews_cleaned))]

# Vérifier la structure du DataFrame
display(df.sample(10))

,critique,film_title
415,autant javais adoré le premier opus de cette t...,"Le Parrain, 2e partie"
331,star wars episode v lempire contreattaque es...,Star Wars : Episode V - L'Empire contre-attaque
697,cest toujours aussi dingue visuellement le mél...,Spider-Man : Across The Spider-Verse
566,trois étoiles peu généreuses ce film est vraim...,Les Temps modernes
515,en septembre dans sa conclusion de son excell...,Dune : DeuxiÃ¨me Partie
245,a lheure où bon nombre veulent en découdre con...,Le Comte de Monte-Cristo
294,cette trilogie est ma préfèrée les deux tours ...,Le Seigneur des anneaux : les deux tours
56,la justice possède sa propre mythologie audelà...,12 hommes en colÃ¨re
21,schindlers list ou le meilleur film de steven ...,La Liste de Schindler
39,un drame fantastique puissant qui ne peut lais...,La Ligne verte


In [9]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Modèle déjà fine-tuné pour l'analyse de sentiment en français
model_name = "tblard/tf-allocine"  # Modèle fine-tuné sur les critiques Allociné !
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, from_tf=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

All TF 2.0 model weights were used when initializing CamembertForSequenceClassification.

All the weights of CamembertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use CamembertForSequenceClassification for predictions without further training.


In [10]:
# Définir le device (GPU si disponible, sinon CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assurez-vous que le modèle est aussi sur le bon device
model = model.to(device)

def split_review(review, max_length=512):
    # Tokenisation de la critique
    tokens = tokenizer.tokenize(review)
    
    # Diviser les tokens en morceaux de taille 'max_length'
    chunks = [tokens[i:i+max_length] for i in range(0, len(tokens), max_length)]
    
    # Recomposer les morceaux en texte
    return [tokenizer.convert_tokens_to_string(chunk) for chunk in chunks]

In [11]:
def get_sentiment_for_review(review):
    # Diviser la critique en morceaux
    chunks = split_review(review)
    sentiments = []  # Liste pour stocker les scores de sentiment de chaque chunk
    
    # Analyser chaque morceau
    for chunk in chunks:
        # Tokenisation et envoi sur le device (GPU/CPU)
        inputs = tokenizer(chunk, truncation=True, padding=True, max_length=512, return_tensors='pt').to(device)
        
        with torch.no_grad():
            outputs = model(**inputs)  # Obtenir les logits du modèle
            
        # Appliquer softmax pour obtenir les probabilités
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        
        # Ajouter le score de sentiment pour ce chunk
        sentiment_score = probs[0, 1].item()  # Classe positive (si modèle binaire)
        sentiments.append(sentiment_score)
    
    # Calculer la moyenne des sentiments de tous les chunks
    if sentiments:
        final_sentiment = sum(sentiments) / len(sentiments)
    else:
        final_sentiment = 0.5  # Valeur par défaut si pas de chunks
        
    return final_sentiment

In [12]:
df['sentiment'] = df['critique'].apply(get_sentiment_for_review)



In [13]:
# Voir le DataFrame avec les sentiments
print(df[['film_title', 'critique', 'sentiment']].sample(10))

                        film_title  \
20           La Liste de Schindler   
431                     Fight Club   
311                          Seven   
390                       Parasite   
184  Le Bon, la brute et le truand   
300                          Seven   
671             American History X   
406          Le Parrain, 2e partie   
667             American History X   
627         Le Silence des agneaux   

                                              critique  sentiment  
20   incroyable film sur oskar shindler cet homme n...   0.998835  
431  une oeuvre étrange cruelle et violente signée ...   0.997108  
311  cest vraiment un excellent thriller  lhistoire...   0.998745  
390  le scénario est très bien écrit et beaucoup pl...   0.998492  
184  si la trame de fond du bon la bruteet le truan...   0.996877  
300  le format et lidée de base sont des plus class...   0.998336  
671  aïe plus les années passent et moins je parvie...   0.078688  
406  comme son prédecesseur le parrai

In [14]:
positive_review = "Ce film est absolument génial, j'ai adoré!"
negative_review = "Un des pires films que j'ai vus, ennuyeux et mal joué"

print(f"Score positif: {get_sentiment_for_review(positive_review)}")
print(f"Score négatif: {get_sentiment_for_review(negative_review)}")

Score positif: 0.9985256791114807
Score négatif: 0.0008790758438408375


In [16]:
df_sorted = df.sort_values(by='sentiment', ascending=True)

# Afficher les 10 lignes ayant les moindres sentiments
print(df_sorted[['film_title', 'critique', 'sentiment']].head(10))

                               film_title  \
696  Spider-Man : Across The Spider-Verse   
701  Spider-Man : Across The Spider-Verse   
533  NapolÃ©on vu par Abel Gance partie 2   
251              Le Comte de Monte-Cristo   
613               Le Tombeau des lucioles   
372  NapolÃ©on vu par Abel Gance partie 1   
415                 Le Parrain, 2e partie   
74                             Le Parrain   
640                              Whiplash   
624                Le Silence des agneaux   

                                              critique  sentiment  
696  là où le premier était vraiment une révélation...   0.000664  
701  bon je ne vais pas être tendre et je sens que ...   0.000837  
533  légèrement inférieure à la première partie ici...   0.001174  
251  alors pour ceux qui nont pas lu le livre ou qu...   0.001333  
613  la singularité du tombeau des lucioles réside ...   0.001660  
372  présenté comme lun des plus grands chefsdœuvre...   0.002223  
415  autant javais adoré le 

# Generation des résumés des critiques des films 

In [39]:
grouped_reviews = df.groupby('film_title')['critique'].apply(lambda x: ' '.join(x)).reset_index()
print(f"Nombre de films: {len(grouped_reviews)}")

Nombre de films: 50


In [40]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from tqdm.notebook import tqdm


In [42]:
model_name = "facebook/mbart-large-50-many-to-many-mmt"  # Modèle français de résumé
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

c:\nlp\nlp\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Jihed B\.cache\huggingface\hub\models--facebook--mbart-large-50-many-to-many-mmt. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [43]:
# 4. Vérifier si GPU est disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(f"Utilisation de: {device}")

Utilisation de: cpu


In [44]:
tokenizer.src_lang = "fr_XX"  # Français comme langue source
tokenizer.tgt_lang = "fr_XX"  # Français comme langue cible


In [ ]:
def split_text_into_segments(text, max_length=300):  # Réduit à 300 pour CPU
    # Diviser par phrases pour une segmentation plus logique
    sentences = [s + '.' for s in text.split('.') if s.strip()]
    
    segments = []
    current_segment = []
    current_length = 0
    
    for sentence in sentences:
        
        sentence_length = len(tokenizer.tokenize(sentence))
        if current_length + sentence_length > max_length and current_segment:
            # Joindre le segment actuel et le stocker
            segments.append(' '.join(current_segment))
            current_segment = [sentence]
            current_length = sentence_length
        else:
            current_segment.append(sentence)
            current_length += sentence_length
    
   
    if current_segment:
        segments.append(' '.join(current_segment))
    
    return segments

In [ ]:
def generate_summary(text, max_length=100, min_length=30):  #gpu non dispo donc reduit les params 
    
    if len(tokenizer.tokenize(text)) < max_length * 1.2:
        return text
        
    segments = split_text_into_segments(text)
    segment_summaries = []
    
    for segment in segments:
        inputs = tokenizer(segment, max_length=512, truncation=True, return_tensors="pt").to(device)
        
        try:
            
            summary_ids = model.generate(
                inputs["input_ids"],
                max_length=max_length,
                min_length=min_length,
                length_penalty=1.5,
                num_beams=2,  
                early_stopping=True,
                forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"]  
            )
            
            segment_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            segment_summaries.append(segment_summary)
        except Exception as e:
            print(f"Erreur lors du traitement d'un segment: {e}")
            # Utiliser une approche de secours en extrayant la première phrase
            first_sentence = segment.split('.')[0] + '.'
            segment_summaries.append(first_sentence)
    
    # Joindre tous les résumés des segments
    full_summary = ' '.join(segment_summaries)
    return full_summary

In [51]:
from tqdm import tqdm
tqdm.pandas()
grouped_reviews['summary'] = grouped_reviews['critique'].progress_apply(generate_summary)

# 8. Ajouter une fonction pour extraire les mots-clés
from sklearn.feature_extraction.text import TfidfVectorizer
import heapq
import nltk
from nltk.corpus import stopwords

# Télécharger les stopwords français si nécessaire
try:
    french_stopwords = stopwords.words('french')
except:
    nltk.download('stopwords')
    french_stopwords = stopwords.words('french')

# Ajouter des mots spécifiques aux critiques de films
french_stopwords += ["film", "cinéma", "voir", "bien", "très", "tout", "plus", "peut", "fait", "deux"]

def extract_keywords(text, n=8):
    # Utiliser TF-IDF pour extraire les mots-clés
    vectorizer = TfidfVectorizer(stop_words=french_stopwords, ngram_range=(1, 2))
    
    try:
        tfidf_matrix = vectorizer.fit_transform([text])
        feature_names = vectorizer.get_feature_names_out()
        
        # Obtenir les scores TF-IDF
        scores = tfidf_matrix.sum(axis=0).A1
        
        # Créer un dictionnaire {mot: score}
        word_scores = {feature_names[i]: scores[i] for i in range(len(feature_names))}
        
        # Extraire les n mots avec les scores les plus élevés
        top_keywords = heapq.nlargest(n, word_scores, key=word_scores.get)
        return top_keywords
    except:
        # Fallback simple si TF-IDF échoue
        words = [w for w in text.lower().split() if w not in french_stopwords and len(w) > 3]
        word_freq = {}
        for word in words:
            word_freq[word] = word_freq.get(word, 0) + 1
        return heapq.nlargest(n, word_freq, key=word_freq.get)

# 9. Ajouter des mots-clés pour chaque film
grouped_reviews['keywords'] = grouped_reviews['summary'].progress_apply(
    lambda x: extract_keywords(x))

# 10. Fonction pour afficher les résultats
def print_film_summary(film_index):
    film = grouped_reviews.iloc[film_index]
    print(f"Film: {film['film_title']}")
    print(f"Mots-clés: {', '.join(film['keywords'])}")
    print(f"Résumé ({len(film['summary'])} caractères):")
    print(film['summary'])
    print("\n" + "="*80 + "\n")

# Afficher quelques exemples
for i in range(min(5, len(grouped_reviews))):
    print_film_summary(i)























































































































































100%|██████████| 50/50 [12:21<00:00, 14.83s/it]



100%|██████████| 50/50 [00:00<00:00, 628.86it/s]

Film: 12 hommes en colÃ¨re
Mots-clés: ans, ans tous, après, après midi, borré, borré suspense, catégorie, catégorie films
Résumé (441 caractères):
ans et tous ses dents men en colère fait partie de cette catégorie de films qui, malgré le temps qui passe et les techniques cinématographiques qui change ne perdent pas de leur impact initial tout premier film du regret sidney lumet qui signera par la suite une après-midi de chien serpico le crime de lorient express h ce samedi ce dram psychologique filmé comme un thriller borré de suspense est intense très intelligent et fait douter le


Film: American History X
Mots-clés: amérique, amérique gangrénée, arbore, arbore croix, are, are two, aîné, aîné donner
Résumé (359 caractères):
Derek et danny are two brothers in the heart dune amérique gangrénée par le racisme le premier arbore une croix gammée sur la poitrine il est violent torturé et prêt à tout pour noyer sa colère le second est larchétype du jeune paumé qui suit le modèle de son frèr

# Extension : Utilisation d'un LLM

In [52]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from tqdm import tqdm
import pandas as pd

# Charger les variables d'environnement du fichier .env
load_dotenv()

# Récupérer la clé API depuis les variables d'environnement
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("La clé API Gemini n'a pas été trouvée dans le fichier .env")

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.2)

In [53]:
prompt_template = ChatPromptTemplate.from_template("""
Tu es un critique de cinéma expert. Voici plusieurs critiques du film "{film_title}". 

Crée un résumé concis (environ 150 mots) qui capture l'essentiel de ces critiques.
Mentionne les points forts, les points faibles, et l'impression générale des spectateurs.
Identifie 3-5 thèmes ou aspects principaux du film selon ces critiques.

Critiques:
{critiques}

Résumé:
""")

In [ ]:
def generate_summary_with_langchain(film_title, critiques_list):
    """Génère un résumé des critiques d'un film en utilisant LangChain avec Gemini Pro 1.5"""
    
    # Joindre les critiques en un seul texte
    critiques_text = "\n\n".join(critiques_list)
    
    # Créer le message à partir du template
    chain = prompt_template | llm
    
    try:
        # Invoquer la chaîne avec les variables
        response = chain.invoke({"film_title": film_title, "critiques": critiques_text})
        return response.content
    except Exception as e:
        print(f"Erreur lors de la génération du résumé pour {film_title}: {e}")
        return f"Erreur: {str(e)}"

# Regrouper les critiques par film
films = df.groupby('film_title')['critique'].apply(list).reset_index()

# Sélectionner un seul film 
selected_film_index = 0  
selected_film = films.iloc[selected_film_index]

film_title = selected_film['film_title']
critiques = selected_film['critique']

print(f"Génération du résumé pour le film: {film_title}")
print(f"Nombre de critiques: {len(critiques)}")

# Générer le résumé pour ce film uniquement
resume = generate_summary_with_langchain(film_title, critiques)

print("\nRésumé généré:")
print("=" * 80)
print(resume)
print("=" * 80)

# Sauvegarder le résultat dans un fichier texte
with open(f"résumé_{film_title.replace(':', '_')}.txt", "w", encoding="utf-8") as f:
    f.write(f"Film: {film_title}\n\n")
    f.write(resume)

print(f"\nLe résumé a été sauvegardé dans le fichier 'résumé_{film_title.replace(':', '_')}.txt'")

Génération du résumé pour le film: 12 hommes en colÃ¨re
Nombre de critiques: 15

Résumé généré:
"12 Hommes en colère" est unanimement salué pour son scénario brillant et ses dialogues percutants, offrant une exploration captivante du système judiciaire et du doute raisonnable. Le huis clos, magistralement mis en scène par Sidney Lumet, renforce la tension palpable et l'atmosphère claustrophobe, immergeant le spectateur au cœur des délibérations.  Les performances des acteurs, notamment Henry Fonda, sont louées pour leur réalisme et leur intensité. 

Malgré quelques critiques pointant un manque de subtilité et une théâtralité excessive par moments, le film est considéré comme un classique indémodable, pertinent encore aujourd'hui.  Sa conclusion, bien que parfois jugée prévisible, n'enlève rien à la puissance du message.

Thèmes principaux:

* **Le système judiciaire et ses failles:** Le film explore les faiblesses du système, les préjugés des jurés, et l'importance du doute raisonnable